In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt

In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 30   # EarlyStopping pehle hi rok dega

TRAIN_DIR = r"C:\Users\shikh\AIML\plant\plant_disease_prediction\datasets\CORN\train"
TEST_DIR = r"C:\Users\shikh\AIML\plant\plant_disease_prediction\datasets\CORN\test"


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
train_data = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


Found 3348 images belonging to 4 classes.
Found 840 images belonging to 4 classes.


In [5]:
class_names = list(train_data.class_indices.keys())
class_names

['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']

In [11]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),   # Reduced dropout
    layers.Dense(4, activation='softmax')
])


In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 26, 26, 128)     

In [14]:
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# 🔹 folder jahan model save hoga
SAVE_DIR = r"C:\Users\shikh\AIML\plant\plant_disease_prediction\trained_models"
os.makedirs(SAVE_DIR, exist_ok=True)

# 🔹 full model path

MODEL_PATH = os.path.join(SAVE_DIR, "corn_best_model.h5")

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),

    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=2
    ),

    ModelCheckpoint(
        filepath=MODEL_PATH,
        monitor='val_loss',
        save_best_only=True
    )
]



In [15]:
history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=test_data,
    callbacks=callbacks
)


Epoch 1/30
105/105 [==============================] - 120s 1s/step - loss: 0.8156 - accuracy: 0.6723 - val_loss: 0.5415 - val_accuracy: 0.7238 - lr: 1.0000e-04
Epoch 2/30
105/105 [==============================] - 127s 1s/step - loss: 0.4946 - accuracy: 0.8026 - val_loss: 0.4308 - val_accuracy: 0.8131 - lr: 1.0000e-04
Epoch 3/30
105/105 [==============================] - 129s 1s/step - loss: 0.4613 - accuracy: 0.8073 - val_loss: 0.4272 - val_accuracy: 0.8095 - lr: 1.0000e-04
Epoch 4/30
105/105 [==============================] - 132s 1s/step - loss: 0.4332 - accuracy: 0.8157 - val_loss: 0.4130 - val_accuracy: 0.8214 - lr: 1.0000e-04
Epoch 5/30
105/105 [==============================] - 140s 1s/step - loss: 0.4201 - accuracy: 0.8247 - val_loss: 0.3743 - val_accuracy: 0.8464 - lr: 1.0000e-04
Epoch 6/30
105/105 [==============================] - 126s 1s/step - loss: 0.4074 - accuracy: 0.8220 - val_loss: 0.3709 - val_accuracy: 0.8357 - lr: 1.0000e-04
Epoch 7/30
105/105 [====================